In [1]:
import meep as mp
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
from numpy import cos, sin
import json

# create geometric and write it to a file

generate random numbers

In [10]:
np.random.seed(15)

num_crystal = 190

a = 5. # shape

size_cell_l = [2, 2, 2]
size_solid_l = [1, 1, 1]
size_crystal_base = [0.1, 0.1, 0.1]
size_crystal_l = np.random.weibull(a, (num_crystal, 3))
for i in range(3):
    size_crystal_l[:, i] *= size_crystal_base[i]

size_cell = mp.Vector3(*size_cell_l)
size_solid = mp.Vector3(*size_solid_l)

size_crystal = []
for i in range(num_crystal):
    size_crystal.append(mp.Vector3(*size_crystal_l[i, :]))

solid_region = mp.Block(size_solid, 
                    center = mp.Vector3(0, 0, 0),
                    material=mp.Medium(epsilon=6))

mean= (0, 0, 0)
cov = [[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]]
loc = np.random.multivariate_normal(mean, cov, (num_crystal))
loc = np.random.uniform(-size_solid_l[0]/2, size_solid_l[0]/2, (num_crystal, 3))
theta_x = np.random.uniform(0, 2*np.pi, num_crystal)
theta_y = np.random.uniform(0, 2*np.pi, num_crystal)
theta_z = np.random.uniform(0, 2*np.pi, num_crystal)

In [11]:
R = np.empty((num_crystal, 3, 3))

Rx_matrix = np.empty((num_crystal, 3, 3))
Ry_matrix = np.empty((num_crystal, 3, 3))
Rz_matrix = np.empty((num_crystal, 3, 3))

for i in range(num_crystal):
    Rx_matrix[i, :, :] = np.array([[1, 0, 0],
                   [0, cos(theta_x[i]), -sin(theta_x[i])], 
                  [0, sin(theta_x[i]), cos(theta_x[i])]])
    
    Ry_matrix[i, :, :] = np.array([[cos(theta_y[i]), 0, sin(theta_y[i])], 
                  [0, 1, 0],
                  [-sin(theta_y[i]), 0, cos(theta_y[i])]])
    
    Rz_matrix[i, :, :] = np.array([[cos(theta_z[i]), -sin(theta_z[i]), 0],
                 [sin(theta_z[i]), cos(theta_z[i]), 0],
                 [0, 0, 1]])

    R[i, :, :] = np.matmul(np.matmul(Ry_matrix[i, :, :], Rx_matrix[i, :, :]), Rz_matrix[i, :, :])


og_x = np.array([[1, 0, 0] for i in range(num_crystal)])
og_y = np.array([[0, 1, 0] for i in range(num_crystal)])
og_z = np.array([[0, 0, 1] for i in range(num_crystal)])

Rx_vector = np.empty((num_crystal, 3))
Ry_vector = np.empty((num_crystal, 3))
Rz_vector = np.empty((num_crystal, 3))

for i in range(num_crystal):
    Rx_vector[i, :] = np.matmul(R[i, :, :], og_x[i, :])
    Ry_vector[i, :] = np.matmul(R[i, :, :], og_y[i, :])
    Rz_vector[i, :] = np.matmul(R[i, :, :], og_z[i, :])



In [52]:
export_geo = True

if export_geo:
    file_name = 'geometry.peter'
    to_write = [num_crystal, size_solid_l, size_crystal_l, loc, theta_x, theta_y, theta_z]
    for i in range(len(to_write)):
        if type(to_write[i]) is not int and type(to_write[i]) is not list:
            to_write[i] = to_write[i].tolist()
    with open(file_name, 'w') as f:
        json.dump(to_write, f)

#\\ad.monash.edu\home\User045\dche145\Documents\Abaqus\geometry_shapes
#\\Client\D$\source\working_with_meep

Create the visualization for visual inspection

In [24]:
pml_layers = [mp.PML(0.3)]

geometry = [mp.Block((1.0,1.0,1.0), 
                    center = mp.Vector3(0, 0, 0),
                    material=mp.Medium(epsilon=1))
,]

geometry.append(mp.Block(
            mp.Vector3(0.2, 0.2, 0.2),
            center = mp.Vector3(0.0, 0.1, 0.2),
            material=mp.Medium(epsilon=10.5)))
source_pad = 0.25
source = [mp.Source(mp.ContinuousSource(wavelength=2*(11**0.5), width=20),
                   component=mp.Ez,
                   center=mp.Vector3(0.75, 0, 0),
                   size=mp.Vector3(0, 0.1, 0.1))]
sim = mp.Simulation(resolution=100,
                    cell_size=size_cell,
                    boundary_layers=pml_layers,
                    sources = source,
                    geometry=geometry)
sim.init_sim()
eps_data = sim.get_epsilon()

show_3d_geo = False
if show_3d_geo:
    from mayavi import mlab
    s = mlab.contour3d(eps_data, colormap="YlGnBu")
    mlab.show()

-----------
Initializing structure...
time for choose_chunkdivision = 0.000535011 s
Working in 3D dimensions.
Computational cell is 2 x 2 x 2 with resolution 100
block, center = (0,0,0)
          size (1,1,1)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
block, center = (0,0.1,0.2)
          size (0.2,0.2,0.2)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (10.5,10.5,10.5)
time for set_epsilon = 14.0367 s
-----------


cheeky run of the geometry

In [65]:
points = np.array([[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2],
                   [2, 0], [2, 1], [2, 2]])
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(points)
import matplotlib.pyplot as plt
voronoi_plot_3d(vor)
plt.show()

NameError: name 'voronoi_plot_3d' is not defined

In [23]:
sim.run(mp.at_beginning(mp.output_epsilon), 
       mp.to_appended("row_col_test", mp.at_every(0.6, mp.output_efield_z)),
       until=0.1)

creating output file "./eps-000000200.h5"...
creating output file "./row_col_test.h5"...
on time step 200 (time=1), 0.445471 s/step
Meep progress: 0.040000000000000036/2.0 = 2.0% done in 4.0s, 197.3s to go
on time step 216 (time=1.08), 0.252211 s/step
Meep progress: 0.1200000000000001/2.0 = 6.0% done in 8.1s, 127.4s to go
on time step 232 (time=1.16), 0.253827 s/step
Meep progress: 0.19999999999999996/2.0 = 10.0% done in 12.2s, 109.4s to go
on time step 249 (time=1.245), 0.245319 s/step
Meep progress: 0.2849999999999999/2.0 = 14.2% done in 16.2s, 97.8s to go
on time step 266 (time=1.33), 0.240982 s/step
Meep progress: 0.3700000000000001/2.0 = 18.5% done in 20.4s, 89.8s to go
on time step 283 (time=1.415), 0.243934 s/step
Meep progress: 0.45500000000000007/2.0 = 22.8% done in 24.5s, 83.2s to go
on time step 300 (time=1.5), 0.242757 s/step
Meep progress: 0.54/2.0 = 27.0% done in 28.7s, 77.5s to go
on time step 316 (time=1.58), 0.255226 s/step
Meep progress: 0.6000000000000001/2.0 = 30.0%

visulization of the output field

In [38]:
# ez_data = sim.get_array(center=mp.Vector3(), size=size_cell, component=mp.Ez)
# print(ez_data.shape)
get_gif_out = False

if get_gif_out:
    !h5ls ez.h5
    !mkdir temp
    !mv eps-000000.00.h5 temp/eps-000000.00.h5
    !mv ez.h5 temp/ez.h5
    import os
    os.chdir('temp')
    !h5topng -z 20 -t 0:332 -R -Zc dkbluered -a yarg -A eps-000000.00.h5 ez.h5
    !convert ez.t*.png ez.gif
    os.chdir('..')

(100, 100, 100)
ez                       Dataset {100, 100, 100, 333/Inf}
h5topng error: invalid colormap file
